In [ ]:
pip install azureml-core

In [1]:
from azureml.core import Workspace
ws = Workspace.from_config(path = "config.json")
print(ws)

Workspace.create(name='MSA-model-deployment', subscription_id='8f613df3-30f7-40d7-a9a3-8861f9369752', resource_group='MSA-model-deployment')


In [2]:
from azureml.core.model import Model

# Register model
#model = Model.register(ws, model_name = "lit-regressor", model_path = "finalized_model.sav")
model = Model(ws, id='lit-regressor:1')

In [4]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig


# Create environment 
env = Environment(name = "lit-regressor")
conda_dep = CondaDependencies()
conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("xgboost")
conda_dep.add_conda_package("pandas")
conda_dep.add_conda_package("scikit-learn")
env.python.conda_dependencies = conda_dep

dummy_inference_config = InferenceConfig(
    environment = env,
    source_directory = "./source_dir",
    entry_script = "./echo_score.py",
)

In [24]:
pip install azure-ml-api-sdk

Note: you may need to restart the kernel to use updated packages.


In [5]:
from azureml.core.webservice import AciWebservice
aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(
    ws,
    "lit-regressor",
    [model],
    dummy_inference_config,
    aci_config,
    overwrite = True,
)
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-10-15 08:51:15+13:00 Creating Container Registry if not exists.
2021-10-15 08:51:15+13:00 Registering the environment.
2021-10-15 08:51:17+13:00 Use the existing image.
2021-10-15 08:51:18+13:00 Generating deployment configuration.
2021-10-15 08:51:18+13:00 Submitting deployment to compute..
2021-10-15 08:51:23+13:00 Checking the status of deployment lit-regressor..
2021-10-15 08:55:16+13:00 Checking the status of inference endpoint lit-regressor.
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 0e03df40-08ff-4849-a13e-ab9f127e10cd
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: lit-regressor. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image msamodeldeployment.azurecr.io/azureml/azureml_81c20017cb737c0aabf198e9e484b0bf locally. Please refer to https://aka.ms/debugimage#service-launch-fails for more inf

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 0e03df40-08ff-4849-a13e-ab9f127e10cd
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: lit-regressor. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image msamodeldeployment.azurecr.io/azureml/azureml_81c20017cb737c0aabf198e9e484b0bf locally. Please refer to https:/

In [ ]:
import requests
import json

uri = service.scoring_uri
requests.get(uri)
headers = {"Content-Type": "application/json"}
data = pd.DataFrame.from_dict({"excerpt": ['How are you', 'It is 3 am in the morning', 
                                           'Human will die eventually, only their emotion remains, passed down and be eternal']})
data = json.dumps(data)
response = requests.post(uri, data = data, headers = headers)
print(response.json())